In [1]:
# ~~~ Imports ~~~
import subprocess
import os
import hydra
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.backends.api_info import ApiInfo
from aiflows.utils.general_helpers import find_replace_in_dict
from aiflows.flow_launchers import FlowLauncher
from aiflows.backends.api_info import ApiInfo
from aiflows import flow_verse
from aiflows import logging

logging.set_verbosity_info()

/Users/nicolasbaldwin/opt/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining Functions Which Reproduce The Steps Of The Quick Start Tutorial (no modifications required)

In [9]:
# ~~~ Convenience function that install the dependencies for the Flows in this notebook ~~~
# Normally, you would install the dependencies in your terminal, as necessary. We do it here for you convenience

DEPENDENCIES_DICT = {
    "ChatFlowModule": "aiflows/ChatFlowModule",
    "ReAct": "aiflows/ControllerExecutorFlowModule",
    "ChatInteractiveFlowModule": "aiflows/ChatInteractiveFlowModule",
    "ChatWithDemonstrationsFlowModule": "aiflows/ChatWithDemonstrationsFlowModule",
    "AutoGPTFlowModule": "aiflows/AutoGPTFlowModule",
    "VisionFlowModule": "aiflows/VisionFlowModule",
}

def download_all_flow_dependencies(flow):
    """Download all dependencies for a flow from FlowVerse and pip requirements. 
    
    Note that this is a convenience function for this tutorial and is not meant to work doesn't work for all Flows on FlowVerse.
    
    :param flow: The name of the Flow to download dependencies for
    :type flow: str
    """
    # ~~~ Load Flow dependecies from FlowVerse ~~~
    dependencies = [
        {"url": DEPENDENCIES_DICT[flow], "revision": "main"},
    ]
    flow_verse.sync_dependencies(dependencies)
    
# ~~~ Load Flow dependecies from FlowVerse and pip requirments ~~~
for key in DEPENDENCIES_DICT.keys():
    download_all_flow_dependencies(key)

[2023-12-15 18:06:02,862][aiflows.flow_verse.loading:885][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/examples/quick_start/flow_modules...
[2023-12-15 18:06:03,150][aiflows.flow_verse.loading:710][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-15 18:06:03,300][aiflows.flow_verse.loading:936][INFO] - [<interactive>] finished syncing


[2023-12-15 18:06:03,305][aiflows.flow_verse.loading:885][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/examples/quick_start/flow_modules...
[2023-12-15 18:06:03,532][aiflows.flow_verse.loading:710][INFO] - aiflows/ControllerExecutorFlowModule:main already synced, skip
[2023-12-15 18:06:03,604][aiflows.flow_verse.loading:936][INFO] - [<interactive>] finished syncing


[2023-12-15 18:06:03,611][aiflows.flow_ver

In [3]:
def run_flow(flow, data, api_information):
    """Loads the Flow config, instantiates the Flow and runs inference.
    
    :param flow: The name of the flow to run
    :type flow: str
    :param data: The input data to the flow
    :type data: dict
    :param api_information: The api information for the flow
    :type api_information: ApiInfo
    """
    
    flow_url = DEPENDENCIES_DICT[flow]

    # ~~~ Load Flow config ~~~
    path_to_config = os.path.join( "flow_modules" ,flow_url, "demo.yaml")
    cfg = read_yaml_file(path_to_config)

    # set the API information
    cfg = find_replace_in_dict(cfg, key_to_find="api_infos", new_value= api_information , current_path="")
  
    # ~~~ Instantiate the Flow ~~~
    flow_with_interfaces = {
        "flow": hydra.utils.instantiate(cfg["flow"], _recursive_=False, _convert_="partial"),
        "input_interface": None,
        "output_interface": None
    }
    
    # ~~~ Run inference ~~~
    path_to_output_file = None
    # path_to_output_file = "output.jsonl"  # Uncomment this line to save the output to disk

    _, outputs = FlowLauncher.launch(
        flow_with_interfaces=flow_with_interfaces, data=data, path_to_output_file=path_to_output_file
    )

    # ~~~ Print the output ~~~
    flow_output_data = outputs[0]
    print(flow_output_data)
    

### Setting you API Information

Set your API information here (copy-paste it)

In [4]:

#OpenAI backend
api_key = ""# copy paste your api key here
api_information = [ApiInfo(backend_used="openai", api_key=api_key)]

# Azure backend
# api_key = "" # copy paste your api key here
# api_base = "" # copy paste your api base here
# api_version = "" #copypase your api base here
# api_information = ApiInfo(backend_used = "azure",
#                           api_base =api_base,
#                           api_key = api_version,
#                           api_version =  api_version )

### Pick a Flow, some data and run it!

Set the `Flow` and `data` values, then run the cell! Examples for each flow from the FlowVerse are provided in the cell bellow. Uncomment the flow and data you wish to run. Here is the list of flows available for you to run:

* [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

* [ReAct](https://huggingface.co/aiflows/ControllerExecutorFlowModule)

* [AutoGPTFlowModule](https://huggingface.co/aiflows/AutoGPTFlowModule)

* [ChatInteractiveFlowModule](https://huggingface.co/aiflows/ChatInteractiveFlowModule)

* [ChatWithDemonstrationsFlowModule](https://huggingface.co/aiflows/ChatWithDemonstrationsFlowModule)

* [VisionFlowModule](https://huggingface.co/aiflows/VisionFlowModule)


In [5]:
# Example for a simple chatbot
# [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

flow = "ChatFlowModule"  
data = {"id": 0, "question": "Who was the NBA champion in 2023?"} 
run_flow(flow, data, api_information)

[2023-12-15 18:04:46,386][aiflows.flow_launchers.flow_API_launcher:194][INFO] - Running inference for ID (sample 0): 0
[{'api_output': "I'm sorry, but as an AI language model, I don't have access to real-time information or the ability to predict future events. As of now, I cannot provide you with the answer to who the NBA champion was in 2023. I recommend checking reliable sports news sources or conducting an internet search for the most up-to-date information."}]


In [6]:
# Example for ReAct (the simplest instance of a ControllerExecutorFlowModule)
flow = "ReAct"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-15 18:04:49,625][aiflows.flow_verse.loading:885][INFO] - [flow_modules.aiflows.ControllerExecutorFlowModule] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/examples/quick_start/flow_modules...
[2023-12-15 18:04:49,910][aiflows.flow_verse.loading:710][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-15 18:04:50,048][aiflows.flow_verse.loading:936][INFO] - [flow_modules.aiflows.ControllerExecutorFlowModule] finished syncing


[2023-12-15 18:04:50,052][aiflows.base_flows.circular:94][DEBUG] - output_msg_payload_processor [ControllerExecutorFlow.detect_finish_or_continue] registered
[2023-12-15 18:04:50,065][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/aiflows/base_flows/CompositeFlow.yaml.
[2023-12-15 18:04:50,101][aiflows.base_flows.abstract:158][DEBUG] - Flow 

CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ControllerFlow                                                                                             
├── description
│   └── Proposes the next action to take towards achieving the goal, and prepares the input for the executor.      
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'aiflows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_vers
├── system_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'You are a smart AI assistant. \n\nYour dec
├── init_human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'Here is the goal you need to achieve:\n{{g
├── human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'Here is the response to your last action:\
├── input_interface_initialized
│   └── ['observation']                                                                                            
├── query_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt'}                                                        
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['thought', 'reasoning', 'plan', 'criticism', 'speak', 'command', 'command_args']                          
├── input_interface_non_initialized
│   └── ['goal']                                                                                                   
├── finish
│   └── {'description': 'Signal that the objective has been satisfied, and returns the answer to the user.', 'input
└── commands
    └── {'wiki_search': {'description[2;38;2

[2023-12-15 18:04:50,245][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/aiflows/base_flows/CompositeFlow.yaml.
[2023-12-15 18:04:50,250][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/aiflows/base_flows/AtomicFlow.yaml.
[2023-12-15 18:04:50,264][aiflows.base_flows.abstract:70][DEBUG] - Flow WikiSearchAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── WikiSearchAtomicFlow                                                                                       
├── description
│   └── A Flow that queries the wikipedia API for a page content.                                                  
├── keep_raw_response
│   └── False                                                                                                      
├── lang
│   └── en                                                                                                         
├── top_k_results
│   └── 5                                                                                                          
├── doc_content_chars_max
│   └── 3000                                                                                                       
├── input_interface
│   └── ['search_term']                                                                                            
└── output_interface
    └── ['wiki_content']                                                                                           

[2023-12-15 18:04:50,351][aiflows.base_flows.abstract:70][DEBUG] - Flow unknown_name instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── input_interface
│   └── ['branch', 'branch_input_data']                                                                            
├── output_interface
│   └── ['branch_output_data']                                                                                     
└── subflows_config
    └── {'wiki_search': {'_target_': 'flow_modules.aiflows.ControllerExecutorFlowModule.WikiSearchAtomicFlow.instan

[2023-12-15 18:04:50,392][aiflows.base_flows.abstract:70][DEBUG] - Flow CtrlEx instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 30                                                                                                         
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Select the next action and prepare the input for the executor.', 'input_interface': {'_target_':
├── name
│   └── CtrlEx                                                                                                     
├── description
│   └── ControllerExecutor (i.e., MRKL, ReAct) interaction implementation with Flows that approaches the problem so
├── input_interface
│   └── ['goal']                                                                                                   
├── output_interface
│   └── ['answer', 'status']                                                                                       
└── subflows_config
    └── {'Controller': {'_target_': 'flow_modules.aiflows.ControllerExecutorFlowModule.ControllerAtomicFlow.instant

[2023-12-15 18:04:50,459][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Controller` ~~~
{
    "message_id": "4ad0a540-ff19-4fe3-81f1-571a5bc0f0e8",
    "created_at": "2023-12-15 17:04:50.458862000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Controller"
}
[2023-12-15 18:04:50,462][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `wiki_search` ~~~
{
    "message_id": "ff55e49a-5f01-4a94-ba5f-d6e334224a24",
    "created_at": "2023-12-15 17:04:50.462426000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "wiki_search"
}
[2023-12-15 18:04:50,475][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Executor` ~~~
{
    "message_id": "3e

In [10]:
# Example for AutoGPTFlowModule
flow = "AutoGPTFlowModule"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-15 18:06:58,864][aiflows.flow_verse.loading:885][INFO] - [flow_modules.aiflows.AutoGPTFlowModule] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflowsForks/aiflows-python-version-issue/examples/quick_start/flow_modules...
[2023-12-15 18:06:59,041][aiflows.flow_verse.loading:710][INFO] - aiflows/ControllerExecutorFlowModule:main already synced, skip
[2023-12-15 18:06:59,366][aiflows.flow_verse.loading:710][INFO] - aiflows/HumanStandardInputFlowModule:main already synced, skip
[2023-12-15 18:06:59,605][aiflows.flow_verse.loading:710][INFO] - aiflows/VectorStoreFlowModule:main already synced, skip
[2023-12-15 18:06:59,699][aiflows.flow_verse.loading:936][INFO] - [flow_modules.aiflows.AutoGPTFlowModule] finished syncing


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nicolasbaldwin/opt/miniconda3/envs/test/lib/python3.11/site-packages/hydra/_internal/utils.py", line 644, in _locate
    obj = getattr(obj, part)
          ^^^^^^^^^^^^^^^^^^
AttributeError: module 'flow_modules.aiflows' has no attribute 'AutoGPTFlowModule'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nicolasbaldwin/opt/miniconda3/envs/test/lib/python3.11/site-packages/hydra/_internal/utils.py", line 650, in _locate
    obj = import_module(mod)
          ^^^^^^^^^^^^^^^^^^
  File "/Users/nicolasbaldwin/opt/miniconda3/envs/test/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<froz

In [ ]:
# Example for ChatInteractiveFlowModule
flow = "ChatInteractiveFlowModule"
data = {"id": 0} # you will be able to ask questions once it is running
run_flow(flow, data, api_information)

In [10]:
# Example for ChatWithDemonstrationsFlowModule
flow = "ChatWithDemonstrationsFlowModule"
data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-08 17:13:18,556][aiflows.flow_verse.loading:775][INFO] - [flow_modules.aiflows.ChatWithDemonstrationsFlowModule] started to sync flow module dependencies to /dlabdata1/baldwin/testflows/flows/examples/quick_start/flow_modules...
[2023-12-08 17:13:18,874][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-08 17:13:18,882][aiflows.flow_verse.loading:825][INFO] - [flow_modules.aiflows.ChatWithDemonstrationsFlowModule] finished syncing


[2023-12-08 17:13:18,935][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /dlabdata1/baldwin/testflows/flows/aiflows/base_flows/CompositeFlow.yaml.
[2023-12-08 17:13:18,974][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /dlabdata1/baldwin/testflows/flows/aiflows/base_flows/AtomicFlow.yaml.
[2023-12-08 17:13:20,035][aiflows.base_flows.abstract:70][DEBUG] - Flow DemonstrationAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── DemonstrationAtomicFlow                                                                                    
├── description
│   └── A flow which returns Demonstrations                                                                        
├── data
│   └── [{'query_data': {'query': 'What is the capital of Turkey?'}, 'response_data': {'response': 'Istambul, my si
├── params
│   └── {'data_dir': None, 'demonstrations_id': 'my_sir_demo', 'ids_to_keep': None, 'demonstrations_k': None}      
├── query_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{query}}',
└── response_prompt_template
    └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': '{{response}}', 'input_variables': ['respon

[2023-12-08 17:13:20,084][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /dlabdata1/baldwin/testflows/flows/aiflows/base_flows/AtomicFlow.yaml.
[2023-12-08 17:13:20,482][aiflows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── SimpleQA_Flow                                                                                              
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'aiflows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_vers
├── system_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully 
├── init_human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{question}
├── human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']} 
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt'}                                                        
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── ['question']                                                                                               

[2023-12-08 17:13:20,803][aiflows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow_with_Demonstrations instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 1                                                                                                          
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Get Demonstrations', 'input_interface': {'_target_': 'aiflows.interfaces.KeyInterface'}, 'flow':
├── name
│   └── SimpleQA_Flow_with_Demonstrations                                                                          
├── description
│   └── A sequential flow that answers questions with demonstrations                                               
├── subflows_config
│   └── {'demonstration_flow': {'_target_': 'flow_modules.aiflows.ChatWithDemonstrationsFlowModule.DemonstrationsAt
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── output_interface
│   └── ['answer']                                                                                                 
└── input_interface
    └── ['questions']                                                                                              

[2023-12-08 17:13:20,860][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `demonstration_flow` ~~~
{
    "message_id": "e13d5569-0cde-4ab8-8083-d3c9555ce4ab",
    "created_at": "2023-12-08 16:13:20.860155343",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "demonstration_flow"
}
[2023-12-08 17:13:20,862][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `chat_flow` ~~~
{
    "message_id": "d1819cca-2445-4686-a993-9ea3e2969ebe",
    "created_at": "2023-12-08 16:13:20.862764104",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "chat_flow"
}
[2023-12-08 17:13:20,865][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `SimpleQA_Flow_with_Demons

In [11]:
# Example for VisionFlowModule
flow = "VisionFlowModule"
url_image = {"type": "url",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}
data = {"id": 0, "query": "What’s in this image?", "data": {"images": [url_image]}}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-08 17:13:30,774][aiflows.flow_verse.loading:775][INFO] - [flow_modules.aiflows.VisionFlowModule] started to sync flow module dependencies to /dlabdata1/baldwin/testflows/flows/examples/quick_start/flow_modules...
[2023-12-08 17:13:30,933][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-08 17:13:30,941][aiflows.flow_verse.loading:825][INFO] - [flow_modules.aiflows.VisionFlowModule] finished syncing


[2023-12-08 17:13:31,783][aiflows.base_flows.abstract:158][DEBUG] - Flow config not found at /dlabdata1/baldwin/testflows/flows/aiflows/base_flows/AtomicFlow.yaml.
[2023-12-08 17:13:32,490][aiflows.base_flows.abstract:70][DEBUG] - Flow Demo Vision Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── Demo Vision Flow                                                                                           
├── description
│   └── A flow that, given a textual input, and a set of images and/or videos, generates a textual output.         
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'aiflows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_vers
├── system_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully 
├── init_human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query'], 
├── human_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']} 
├── input_interface_initialized
│   └── ['query', 'data']                                                                                          
├── query_message_prompt_template
│   └── {'_target_': 'aiflows.prompt_template.JinjaPrompt'}                                                        
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
├── input_interface
│   └── ['query', 'data']                                                                                          
└── input_interface_non_initialized
    └── ['question', 'data']                                                                                       

[2023-12-08 17:13:32,558][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Demo Vision Flow` ~~~
{
    "message_id": "6529bc7b-deec-4bb9-b495-21c4d0ee8007",
    "created_at": "2023-12-08 16:13:32.558501397",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Demo Vision Flow"
}
[2023-12-08 17:13:32,561][aiflows.flow_launchers.flow_API_launcher:194][INFO] - Running inference for ID (sample 0): 0
[2023-12-08 17:13:32,563][aiflows.base_flows.abstract:309][DEBUG] - 
 ~~~ InputMessage: `Launcher` --> `Demo Vision Flow` ~~~
{
    "message_id": "871bab41-665c-45cb-8631-53d1fa80c45c",
    "created_at": "2023-12-08 16:13:32.563164226",
    "message_type": "InputMessage",
    "created_by": "Launcher",
    "data": {
        "id": 0,
        "query": "What\u2019s in this image?",
        "data": {
            "images": [
                {
       